In [1]:
%matplotlib inline

import os

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

import keras
import numpy as np
import pandas as pd
import skvideo.io as skv
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# load the data
run = '-benchmark-lrcn-micro'
labelpath = os.path.join('train_labels.csv')
train_labels = pd.read_csv(labelpath, index_col='filename')

In [3]:
train_labels.head()

,bird,blank,cattle,chimpanzee,elephant,forest buffalo,gorilla,hippopotamus,human,hyena,...,other (primate),pangolin,porcupine,reptile,rodent,small antelope,small cat,wild dog,duiker,hog
filename,,,,,,,,,,,,,,,,,,,,,
000libDc84.mp4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003TeGtbkD.mp4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006jFoesFi.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
008uxqP8IN.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0094UxdyyZ.mp4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204130 entries, 000libDc84.mp4 to zzzu2lK8bC.mp4
Data columns (total 24 columns):
bird                   204130 non-null float64
blank                  204130 non-null float64
cattle                 204130 non-null float64
chimpanzee             204130 non-null float64
elephant               204130 non-null float64
forest buffalo         204130 non-null float64
gorilla                204130 non-null float64
hippopotamus           204130 non-null float64
human                  204130 non-null float64
hyena                  204130 non-null float64
large ungulate         204130 non-null float64
leopard                204130 non-null float64
lion                   204130 non-null float64
other (non-primate)    204130 non-null float64
other (primate)        204130 non-null float64
pangolin               204130 non-null float64
porcupine              204130 non-null float64
reptile                204130 non-null float64
rodent                 2041

In [5]:
train_labels.sum(axis=0).sort_values(ascending=False)

blank                  122270.0
duiker                  21601.0
other (primate)         20453.0
human                   20034.0
chimpanzee               5045.0
hog                      4650.0
rodent                   2911.0
bird                     2386.0
other (non-primate)      1883.0
elephant                 1085.0
porcupine                 569.0
cattle                    372.0
small antelope            273.0
large ungulate            224.0
leopard                   209.0
hippopotamus              175.0
gorilla                   174.0
small cat                  79.0
pangolin                   63.0
wild dog                   21.0
hyena                      10.0
forest buffalo              9.0
reptile                     8.0
lion                        2.0
dtype: float64

In [6]:
(train_labels.sum(axis=1) > 1).sum()

374

In [7]:
from primatrix_dataset_utils import Dataset

In [8]:
datapath = os.path.join('.')
whichset = run.split('-')[-1]
redframes = whichset == 'nano'
data = Dataset(datapath=datapath, 
               dataset_type=whichset,
               reduce_frames=redframes, 
               batch_size=32, 
               test=False)

In [9]:
data.num_classes

24

In [10]:
data.num_frames

30

In [11]:
data.batch_size

32

In [12]:
data.num_samples

142891

In [13]:
data.y_train.shape[0]

142891

In [14]:
data.y_val.shape[0]

61239

In [15]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/gpu:0']


In [16]:
from keras.models import Sequential
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint

In [18]:
# instantiate model
model = Sequential()

# add three time-distributed convolutional layers for feature extraction
model.add(BatchNormalization(input_shape=(data.num_frames, data.width, data.height, 1)))
model.add(TimeDistributed(Conv2D(64, (5, 5), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(128, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(256, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(512, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(1024, (2,2), activation='relu')))


# extract features and dropout 
model.add(TimeDistributed(Flatten()))
#model.add(Dropout(0.2))

# input to LSTM
model.add(LSTM(256, return_sequences=True, dropout=0.0))
model.add(LSTM(64, return_sequences=False, dropout=0.0))
model.add(Dropout(0.5))

# classifier with sigmoid activation for multilabel
model.add(Dense(data.num_classes, activation='sigmoid'))

adam = keras.optimizers.Adam(lr=0.001, decay=0.0)

# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# look at the params before training
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 30, 64, 64, 1)     4         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 60, 60, 64)    1664      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 30, 30, 64)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 30, 30, 64)    256       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 28, 28, 128)   73856     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 14, 14, 128)   0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 14, 14, 128)   512       
__________

In [19]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=int(data.num_batches/20),                  # data.num_batches to train on full set 
    epochs=6000, 
    validation_data=data.val_batches(), 
    validation_steps=int(data.num_val_batches/20),                  # data.num_val_batches to validate on full set
    callbacks=callbacks_list
)

Epoch 1/6000
223/223 [==============================] - 2342s - loss: 0.1407 - val_loss: 0.0930
Epoch 2/6000
223/223 [==============================] - 2285s - loss: 0.1023 - val_loss: 0.0939
Epoch 3/6000
223/223 [==============================] - 2298s - loss: 0.0987 - val_loss: 0.0925
Epoch 4/6000
223/223 [==============================] - 2291s - loss: 0.0956 - val_loss: 0.0898
Epoch 5/6000
 14/223 [>.............................] - ETA: 457s - loss: 0.0977

Exception in thread Thread-3:
Traceback (most recent call last):
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/skvideo/io/ffmpeg.py", line 267, in _read_frame_data
    assert len(arr) == framesize
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/keras/utils/data_utils.py", line 568, in data_generator_task
    generator_output = next(self._generator)
  File "/home/ninja2/Primatrix/primatrix_dataset_utils.py", line 155, in batches
    verbose=verbose)
  File "/home/ninja2/Primatrix/primatrix_dataset_utils.py", line 274, in _get_video_batch
    video = skv.vread(filepath, as_grey=as_grey)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/skvideo/io/io.py", lin

KeyboardInterrupt: 

In [ ]:
# load model
from keras.models import load_model

trained_model = load_model(model_name)

# generate predictions
for batch_num in tqdm(range(data.num_test_batches), total=data.num_test_batches):

    # make predictions on batch
    results = trained_model.predict_proba(next(data.test_batches()), 
                                          batch_size=data.batch_size, 
                                          verbose=0)

    # update submission format dataframe stored in dataset object
    data.update_predictions(results)          
    
data.predictions.to_csv(os.path.join(data.datapath, 'predictions' + run + '.csv'))

In [ ]:
!head -n 5 ./predictions.csv